## 相关性分析

### 1. 处理测试集

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

# 全局变量
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # 选取属性
feanum=len(cols) - 3 # 特征数量
window=5 # 时间窗设置

test_df = pd.read_csv("../data/biClsData/testData4.csv", usecols=cols)
# test_df = pd.read_csv("../data/testData.csv")
test_stocks_code = test_df["kdcode"].unique()
test_stocks_num = len(test_stocks_code)

"测试集股票数量：",test_stocks_num

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Duplicate key in file PosixPath('/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc'), line 400 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


('测试集股票数量：', 3790)

In [2]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(test_stocks_code)):
    stock_i_data = test_df[test_df['kdcode'].isin([stock_i])]
    exec("test_df%s = stock_i_data" % i)

100%|██████████| 3790/3790 [01:00<00:00, 62.19it/s]


In [3]:
testResult = []
for i in tqdm(range(test_stocks_num)): # 遍历训练集所有股票的DataFrame
    exec("testData = test_df%s.values" % i)
    sequence_length = window + 1
    for index in range(len(testData) - sequence_length + 1):
        testResult.append(testData[index: index + sequence_length])

testResult = np.array(testResult)
testResult.shape

100%|██████████| 3790/3790 [00:01<00:00, 2209.06it/s]


(1280967, 6, 21)

In [4]:
X_test = testResult[:, :-1, 2:-1]
X_test = X_test.astype('float64')
print("测试集X：" + str(X_test.shape))

测试集X：(1280967, 5, 18)


In [5]:
Y_test = testResult[:, window, -1]
Y_test.shape

(1280967,)

In [6]:
Y_test = Y_test.astype('float64')

### 2. 载入模型

In [7]:
from keras.models import load_model

model_path = './models/biClsModel_D4.h5'
model = load_model(model_path)

In [8]:
#在测试集上的预测
Y_test_predict=model.predict(X_test)[:,0]
Y_test=Y_test

In [9]:
Y_test_predict

array([0.47898188, 0.4800783 , 0.43983248, ..., 0.44591925, 0.43368402,
       0.4719357 ], dtype=float32)

In [10]:
# pred_df = pd.DataFrame(testResult[:, -1, :2], columns = ['dt', 'kdcode'])
# pred_df
pred_df = pd.DataFrame(testResult[:, -1, :], columns = test_df.columns)
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label
0,2020-01-09,000004.SZ,-0.42695,-0.301397,0.0821609,0.936395,0.92906,0.92308,0.94419,0.877768,...,0.582445,0.0324201,0.400488,2.2571,2.23185,2.24956,2.28828,1.74184,1.43307,1
1,2020-01-10,000004.SZ,-0.377029,-0.196089,0.355327,0.949297,0.940876,0.925481,0.939236,0.887879,...,0.252368,0.0252969,-0.473124,2.31263,2.26507,2.25497,2.2954,1.56974,-1.1601,1
2,2020-01-13,000004.SZ,-0.191322,0.208741,1.37151,0.997977,0.959899,0.94494,0.941434,0.900586,...,1.21931,0.950583,0.291023,2.31263,2.29307,2.26351,2.27789,4.048,3.61678,1
3,2020-01-14,000004.SZ,-0.353569,-0.135347,0.48368,1.00234,0.981865,0.962617,0.943512,0.913367,...,0.668699,0.674002,0.2569,2.37814,2.33341,2.29654,2.29113,1.43206,0.272965,0
4,2020-01-15,000004.SZ,-0.404875,-0.251033,0.202964,0.965388,0.987225,0.967514,0.944128,0.922639,...,0.246439,0.59315,0.0399335,2.38668,2.35808,2.32815,2.29583,1.63858,-3.20734,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,-0.514694,-0.507466,-0.345995,0.386492,0.397377,0.396838,0.425626,0.424501,...,-0.107606,-0.303945,-0.475431,-0.515812,-0.512767,-0.512394,-0.503448,-0.503884,0.0695445,0
1280963,2021-07-27,688600.SH,-0.484503,-0.46134,-0.139655,0.421314,0.40187,0.401931,0.420365,0.423243,...,0.109716,0.230296,-0.576346,-0.513152,-0.51309,-0.513796,-0.505044,-0.328496,0.231815,0
1280964,2021-07-28,688600.SH,-0.503676,-0.492065,-0.270962,0.379582,0.394838,0.397214,0.409845,0.420194,...,-0.384007,-0.323025,-0.840277,-0.509282,-0.51309,-0.512587,-0.507366,-0.252494,-0.533174,0
1280965,2021-07-29,688600.SH,-0.510451,-0.501071,-0.317858,0.392479,0.396832,0.394149,0.402413,0.418703,...,0.0126596,-0.175155,-0.560821,-0.516296,-0.513251,-0.513409,-0.510534,-0.457114,0.0695445,0


In [11]:
pred_df['score'] = Y_test_predict
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
0,2020-01-09,000004.SZ,-0.42695,-0.301397,0.0821609,0.936395,0.92906,0.92308,0.94419,0.877768,...,0.0324201,0.400488,2.2571,2.23185,2.24956,2.28828,1.74184,1.43307,1,0.478982
1,2020-01-10,000004.SZ,-0.377029,-0.196089,0.355327,0.949297,0.940876,0.925481,0.939236,0.887879,...,0.0252969,-0.473124,2.31263,2.26507,2.25497,2.2954,1.56974,-1.1601,1,0.480078
2,2020-01-13,000004.SZ,-0.191322,0.208741,1.37151,0.997977,0.959899,0.94494,0.941434,0.900586,...,0.950583,0.291023,2.31263,2.29307,2.26351,2.27789,4.048,3.61678,1,0.439832
3,2020-01-14,000004.SZ,-0.353569,-0.135347,0.48368,1.00234,0.981865,0.962617,0.943512,0.913367,...,0.674002,0.2569,2.37814,2.33341,2.29654,2.29113,1.43206,0.272965,0,0.455922
4,2020-01-15,000004.SZ,-0.404875,-0.251033,0.202964,0.965388,0.987225,0.967514,0.944128,0.922639,...,0.59315,0.0399335,2.38668,2.35808,2.32815,2.29583,1.63858,-3.20734,0,0.398201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,-0.514694,-0.507466,-0.345995,0.386492,0.397377,0.396838,0.425626,0.424501,...,-0.303945,-0.475431,-0.515812,-0.512767,-0.512394,-0.503448,-0.503884,0.0695445,0,0.561769
1280963,2021-07-27,688600.SH,-0.484503,-0.46134,-0.139655,0.421314,0.40187,0.401931,0.420365,0.423243,...,0.230296,-0.576346,-0.513152,-0.51309,-0.513796,-0.505044,-0.328496,0.231815,0,0.539409
1280964,2021-07-28,688600.SH,-0.503676,-0.492065,-0.270962,0.379582,0.394838,0.397214,0.409845,0.420194,...,-0.323025,-0.840277,-0.509282,-0.51309,-0.512587,-0.507366,-0.252494,-0.533174,0,0.445919
1280965,2021-07-29,688600.SH,-0.510451,-0.501071,-0.317858,0.392479,0.396832,0.394149,0.402413,0.418703,...,-0.175155,-0.560821,-0.516296,-0.513251,-0.513409,-0.510534,-0.457114,0.0695445,0,0.433684


In [12]:
pred_df['dt'] = pd.to_datetime(pred_df['dt'])
pred_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
0,2020-01-09,000004.SZ,-0.42695,-0.301397,0.0821609,0.936395,0.92906,0.92308,0.94419,0.877768,...,0.0324201,0.400488,2.2571,2.23185,2.24956,2.28828,1.74184,1.43307,1,0.478982
1,2020-01-10,000004.SZ,-0.377029,-0.196089,0.355327,0.949297,0.940876,0.925481,0.939236,0.887879,...,0.0252969,-0.473124,2.31263,2.26507,2.25497,2.2954,1.56974,-1.1601,1,0.480078
2,2020-01-13,000004.SZ,-0.191322,0.208741,1.37151,0.997977,0.959899,0.94494,0.941434,0.900586,...,0.950583,0.291023,2.31263,2.29307,2.26351,2.27789,4.048,3.61678,1,0.439832
3,2020-01-14,000004.SZ,-0.353569,-0.135347,0.48368,1.00234,0.981865,0.962617,0.943512,0.913367,...,0.674002,0.2569,2.37814,2.33341,2.29654,2.29113,1.43206,0.272965,0,0.455922
4,2020-01-15,000004.SZ,-0.404875,-0.251033,0.202964,0.965388,0.987225,0.967514,0.944128,0.922639,...,0.59315,0.0399335,2.38668,2.35808,2.32815,2.29583,1.63858,-3.20734,0,0.398201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280962,2021-07-26,688600.SH,-0.514694,-0.507466,-0.345995,0.386492,0.397377,0.396838,0.425626,0.424501,...,-0.303945,-0.475431,-0.515812,-0.512767,-0.512394,-0.503448,-0.503884,0.0695445,0,0.561769
1280963,2021-07-27,688600.SH,-0.484503,-0.46134,-0.139655,0.421314,0.40187,0.401931,0.420365,0.423243,...,0.230296,-0.576346,-0.513152,-0.51309,-0.513796,-0.505044,-0.328496,0.231815,0,0.539409
1280964,2021-07-28,688600.SH,-0.503676,-0.492065,-0.270962,0.379582,0.394838,0.397214,0.409845,0.420194,...,-0.323025,-0.840277,-0.509282,-0.51309,-0.512587,-0.507366,-0.252494,-0.533174,0,0.445919
1280965,2021-07-29,688600.SH,-0.510451,-0.501071,-0.317858,0.392479,0.396832,0.394149,0.402413,0.418703,...,-0.175155,-0.560821,-0.516296,-0.513251,-0.513409,-0.510534,-0.457114,0.0695445,0,0.433684


### 3. 计算`当天开盘价 - 前一天收盘价`

In [13]:
df = pd.read_csv('../data/data2.csv')
df['dt'] = pd.to_datetime(df['dt'])
df = df[['dt', 'kdcode', 'adj_open', 'adj_close']]
df

,dt,kdcode,adj_open,adj_close
0,2016-01-04,000004.SZ,270.239004,243.215104
1,2016-01-05,000004.SZ,220.783500,227.730821
2,2016-01-06,000004.SZ,227.259816,230.674601
3,2016-01-07,000004.SZ,226.671060,207.595366
4,2016-01-08,000004.SZ,206.829983,208.890629
...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000
4122507,2021-07-27,688600.SH,15.640000,15.840000
4122508,2021-07-28,688600.SH,15.800000,15.340000
4122509,2021-07-29,688600.SH,15.510000,15.570000


In [14]:
df = df[df['dt'] >= '2020']
df

,dt,kdcode,adj_open,adj_close
776,2020-01-02,000004.SZ,133.353234,131.469215
777,2020-01-03,000004.SZ,131.351464,131.586966
778,2020-01-06,000004.SZ,130.703832,128.054430
779,2020-01-07,000004.SZ,128.348808,130.115076
780,2020-01-08,000004.SZ,128.996440,128.231057
...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000
4122507,2021-07-27,688600.SH,15.640000,15.840000
4122508,2021-07-28,688600.SH,15.800000,15.340000
4122509,2021-07-29,688600.SH,15.510000,15.570000


In [15]:
res = df['adj_open'].iloc[1:].values - df['adj_close'].iloc[:-1].values
res

array([-0.1177512, -0.883134 ,  0.294378 , ..., -0.04     ,  0.17     ,
       -0.13     ])

In [16]:
res = list(res)
res.append(0)
len(res)

1299899

In [17]:
df['open_close'] = res
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,dt,kdcode,adj_open,adj_close,open_close
776,2020-01-02,000004.SZ,133.353234,131.469215,-0.117751
777,2020-01-03,000004.SZ,131.351464,131.586966,-0.883134
778,2020-01-06,000004.SZ,130.703832,128.054430,0.294378
779,2020-01-07,000004.SZ,128.348808,130.115076,-1.118636
780,2020-01-08,000004.SZ,128.996440,128.231057,1.942895
...,...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000,0.050000
4122507,2021-07-27,688600.SH,15.640000,15.840000,-0.040000
4122508,2021-07-28,688600.SH,15.800000,15.340000,0.170000
4122509,2021-07-29,688600.SH,15.510000,15.570000,-0.130000


### 4. 计算`当天开盘价 - 前一天开盘价`

In [18]:
res = df['adj_open'].iloc[1:].values - df['adj_open'].iloc[:-1].values
res

array([-2.0017704, -0.6476316, -2.355024 , ...,  0.16     , -0.29     ,
       -0.07     ])

In [19]:
res = list(res)
res.append(0)
len(res)

1299899

In [20]:
df['open_open'] = res
df

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,dt,kdcode,adj_open,adj_close,open_close,open_open
776,2020-01-02,000004.SZ,133.353234,131.469215,-0.117751,-2.001770
777,2020-01-03,000004.SZ,131.351464,131.586966,-0.883134,-0.647632
778,2020-01-06,000004.SZ,130.703832,128.054430,0.294378,-2.355024
779,2020-01-07,000004.SZ,128.348808,130.115076,-1.118636,0.647632
780,2020-01-08,000004.SZ,128.996440,128.231057,1.942895,1.177512
...,...,...,...,...,...,...
4122506,2021-07-26,688600.SH,15.530000,15.590000,0.050000,0.110000
4122507,2021-07-27,688600.SH,15.640000,15.840000,-0.040000,0.160000
4122508,2021-07-28,688600.SH,15.800000,15.340000,0.170000,-0.290000
4122509,2021-07-29,688600.SH,15.510000,15.570000,-0.130000,-0.070000


### 5. 合并`df`和`pred_df`

In [21]:
df.set_index(['dt', 'kdcode'],inplace = True)
df

,,adj_open,adj_close,open_close,open_open
dt,kdcode,,,,
2020-01-02,000004.SZ,133.353234,131.469215,-0.117751,-2.001770
2020-01-03,000004.SZ,131.351464,131.586966,-0.883134,-0.647632
2020-01-06,000004.SZ,130.703832,128.054430,0.294378,-2.355024
2020-01-07,000004.SZ,128.348808,130.115076,-1.118636,0.647632
2020-01-08,000004.SZ,128.996440,128.231057,1.942895,1.177512
...,...,...,...,...,...
2021-07-26,688600.SH,15.530000,15.590000,0.050000,0.110000
2021-07-27,688600.SH,15.640000,15.840000,-0.040000,0.160000
2021-07-28,688600.SH,15.800000,15.340000,0.170000,-0.290000


In [22]:
pred_df.set_index(['dt', 'kdcode'], inplace = True)
pred_df

,,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,pct_chg,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
dt,kdcode,,,,,,,,,,,,,,,,,,,,
2020-01-09,000004.SZ,-0.42695,-0.301397,0.0821609,0.936395,0.92906,0.92308,0.94419,0.877768,0.962013,0.582445,0.0324201,0.400488,2.2571,2.23185,2.24956,2.28828,1.74184,1.43307,1,0.478982
2020-01-10,000004.SZ,-0.377029,-0.196089,0.355327,0.949297,0.940876,0.925481,0.939236,0.887879,0.0173832,0.252368,0.0252969,-0.473124,2.31263,2.26507,2.25497,2.2954,1.56974,-1.1601,1,0.480078
2020-01-13,000004.SZ,-0.191322,0.208741,1.37151,0.997977,0.959899,0.94494,0.941434,0.900586,1.21636,1.21931,0.950583,0.291023,2.31263,2.29307,2.26351,2.27789,4.048,3.61678,1,0.439832
2020-01-14,000004.SZ,-0.353569,-0.135347,0.48368,1.00234,0.981865,0.962617,0.943512,0.913367,-0.0505587,0.668699,0.674002,0.2569,2.37814,2.33341,2.29654,2.29113,1.43206,0.272965,0,0.455922
2020-01-15,000004.SZ,-0.404875,-0.251033,0.202964,0.965388,0.987225,0.967514,0.944128,0.922639,-0.759157,0.246439,0.59315,0.0399335,2.38668,2.35808,2.32815,2.29583,1.63858,-3.20734,0,0.398201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-26,688600.SH,-0.514694,-0.507466,-0.345995,0.386492,0.397377,0.396838,0.425626,0.424501,-0.147447,-0.107606,-0.303945,-0.475431,-0.515812,-0.512767,-0.512394,-0.503448,-0.503884,0.0695445,0,0.561769
2021-07-27,688600.SH,-0.484503,-0.46134,-0.139655,0.421314,0.40187,0.401931,0.420365,0.423243,0.620829,0.109716,0.230296,-0.576346,-0.513152,-0.51309,-0.513796,-0.505044,-0.328496,0.231815,0,0.539409
2021-07-28,688600.SH,-0.503676,-0.492065,-0.270962,0.379582,0.394838,0.397214,0.409845,0.420194,-1.22614,-0.384007,-0.323025,-0.840277,-0.509282,-0.51309,-0.512587,-0.507366,-0.252494,-0.533174,0,0.445919


In [23]:
res_df = pd.concat([df, pred_df], axis = 1)

In [24]:
res_df.dropna(inplace = True)
res_df

adj_open   adj_close  open_close  open_open  \
dt         kdcode                                                     
2020-01-09 000004.SZ  130.173952  131.410339    1.059761   2.296148   
           000005.SZ   32.403106   32.811978   -0.102218   0.306654   
           000007.SZ  113.871744  113.751245    0.602496   0.481997   
           000008.SZ  100.508665  101.044711    0.804069   1.340116   
           000010.SZ   37.813581   39.752739    0.430924   2.370082   
...                          ...         ...         ...        ...   
2021-07-30 688586.SH   25.800000   27.690000   11.410000  13.300000   
           688588.SH   17.760000   17.490000   52.060000  51.790000   
           688589.SH   45.580000   43.000000  -13.510000 -16.090000   
           688596.SH   22.800000   23.420000   -7.410000  -6.790000   
           688600.SH   15.440000   15.550000    0.000000   0.000000   

                         volume   turnover       turn      vwap vwap_mean3  \
dt         kdcode                                                            
2020-01-09 000004.SZ   -0.42695  -0.301397  0.0821609  0.936395    0.92906   
           000005.SZ    1.08196 -0.0820406  0.0770493 -0.619805  -0.620659   
           000007.SZ  -0.121316  -0.187661  0.0129432 -0.110924  -0.110317   
           000008.SZ    2.60778   0.536071  -0.071375 -0.573206  -0.573588   
           000010.SZ   0.205465  -0.308047   0.036344 -0.581261  -0.589255   
...                         ...        ...        ...       ...        ...   
2021-07-30 688586.SH   0.366235     1.7393    3.86522   1.32826    1.20881   
           688588.SH  -0.128288   0.134698   0.174545  0.567868   0.542797   
           688589.SH  0.0215264    1.77963    4.10908   2.70922    2.69969   
           688596.SH -0.0431409   0.561691    3.44316   1.06309   0.958064   
           688600.SH   -0.52149  -0.517221  -0.392891  0.386365    0.38519   

                     vwap_mean5  ...  chg_mean5 chg_mean10   adj_open  \
dt         kdcode                ...                                    
2020-01-09 000004.SZ    0.92308  ...  0.0324201   0.400488     2.2571   
           000005.SZ  -0.622662  ...   0.382634   0.690624    -0.1077   
           000007.SZ  -0.109316  ...  -0.157506 -0.0886166    1.86279   
           000008.SZ  -0.574937  ...   0.287631    0.49975    1.53958   
           000010.SZ  -0.594844  ...    1.30899   0.937777  0.0231636   
...                         ...  ...        ...        ...        ...   
2021-07-30 688586.SH    1.26646  ...   0.724787   0.447219   -0.26741   
           688588.SH    0.56827  ...  -0.160845  -0.347901  -0.461875   
           688589.SH     2.5344  ...    3.31967    2.89801   0.211011   
           688596.SH   0.972049  ...   0.363856    1.22016  -0.339972   
           688600.SH   0.391287  ... -0.0654445  -0.494283  -0.517989   

                     open_mean3  open_mean5  open_mean10  high_low close_open  \
dt         kdcode                                                               
2020-01-09 000004.SZ    2.23185     2.24956      2.28828   1.74184    1.43307   
           000005.SZ  -0.108144   -0.112391    -0.122551 -0.189504   0.473913   
           000007.SZ    1.86768      1.8721       1.8944   0.81181  -0.139668   
           000008.SZ    1.54304     1.53847      1.50079  0.429276   0.621317   
           000010.SZ  0.0157398  0.00728825  1.07754e-05  0.466105    2.24763   
...                         ...         ...          ...       ...        ...   
2021-07-30 688586.SH  -0.273053   -0.260179    -0.250599  0.793989    2.19065   
           688588.SH  -0.467791   -0.462773    -0.460815 -0.328496   -0.31295   
           688589.SH   0.147938   0.0957752   0.00967365   1.38446   -2.99041   
           688596.SH  -0.360185   -0.346894    -0.349214  0.501675   0.718626   
           688600.SH  -0.514863    -0.51515    -0.512807 -0.574039   0.127498   

                     label     score  
dt         kdcode                     
202

### 5. 计算相关性

In [25]:
res_df = res_df.astype(float)
# res_df = res_df[['open_close', 'open_open', 'score']]
res_df.corr()

,adj_open,adj_close,open_close,open_open,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,...,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label,score
adj_open,1.000000,0.999287,-0.042253,-0.049569,0.106146,0.417442,-0.012418,0.368127,0.367752,0.366978,...,0.021197,0.031265,1.000000,0.999350,0.998652,0.997063,0.771038,-0.027473,-0.027076,-0.054191
adj_close,0.999287,1.000000,-0.042371,-0.035323,0.107924,0.420134,-0.009586,0.368632,0.367762,0.366898,...,0.031504,0.038381,0.999287,0.998710,0.998048,0.996524,0.774492,0.010287,-0.016262,-0.053255
open_close,-0.042253,-0.042371,1.000000,0.924914,-0.002084,-0.024233,-0.010403,-0.021078,-0.020849,-0.020656,...,-0.001659,-0.002763,-0.042253,-0.041608,-0.041399,-0.040603,-0.041803,-0.002764,0.061350,0.007634
open_open,-0.049569,-0.035323,0.924914,1.000000,0.015619,0.003294,0.018919,-0.015644,-0.020406,-0.021134,...,0.102148,0.068969,-0.049569,-0.048240,-0.047680,-0.046281,-0.006461,0.377618,0.165749,0.016673
volume,0.106146,0.107924,-0.002084,0.015619,1.000000,0.637892,0.248039,-0.273573,-0.276310,-0.278679,...,0.139170,0.139134,0.106146,0.102656,0.099891,0.094745,0.210705,0.046160,-0.005425,-0.167450
turnover,0.417442,0.420134,-0.024233,0.003294,0.637892,1.000000,0.438551,0.268569,0.263370,0.258779,...,0.194216,0.216012,0.417442,0.412075,0.407721,0.399309,0.565217,0.067685,-0.012955,-0.183145
turn,-0.012418,-0.009586,-0.010403,0.018919,0.248039,0.438551,1.000000,0.237166,0.232462,0.228236,...,0.234554,0.280367,-0.012418,-0.016012,-0.018938,-0.024427,0.233798,0.075103,-0.002573,-0.139684
vwap,0.368127,0.368632,-0.021078,-0.015644,-0.273573,0.268569,0.237166,1.000000,0.999290,0.998382,...,0.056289,0.081187,0.368127,0.367154,0.366190,0.363979,0.405454,0.010187,-0.015687,0.000343
vwap_mean3,0.367752,0.367762,-0.020849,-0.020406,-0.276310,0.263370,0.232462,0.999290,1.000000,0.999600,...,0.040764,0.070855,0.367752,0.367475,0.366695,0.364565,0.403015,-0.002895,-0.020468,0.001815
vwap_mean5,0.366978,0.366898,-0.020656,-0.021134,-0.278679,0.258779,0.228236,0.998382,0.999600,1.000000,...,0.024781,0.060309,0.366978,0.367107,0.366761,0.364949,0.400903,-0.005280,-0.020539,0.005103
